In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Cargar datos (ajusta la ruta y nombre del archivo según corresponda)
df_train = pd.read_csv("../data/train_reviews.csv")
df_train

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,ZZO43qKB-s65zplC8RfJqw,-1BSu2dt_rOAqllw9ZDXtA,smkZq4G1AOm4V6p3id5sww,5.0,0,0,0,Fantastic fresh food. The greek salad is amazi...,2016-09-30 15:49:32
1,vojXOF_VOgvuKD95gCO8_Q,xpe178ng_gj5X6HgqtOing,96_c_7twb7hYRZ9HHrq01g,1.0,2,0,1,Been a patient at Largo Med/Diagnostic Clinic ...,2020-12-09 14:39:51
2,KwxdbiseRlIRNzpgvyjY0Q,axbaerf2Fk92OB4b9_peVA,e0AYjKfSF0DL-5C1CpOq6Q,4.0,0,0,0,The location is convenient to my campus so I d...,2013-09-04 16:19:51
3,3mwoBcTy-2gMh0L91uaIeA,_GOiybb0rImYKJfwyxEaGg,vF-uptiQ34pVLHJKzPHUlA,5.0,0,0,0,I agree with all the other compliments posted ...,2019-03-02 12:24:14
4,XfWf7XsBWs3kYyYq7Ns1ZQ,ojWKg3B5pH3ncAsxun3kUw,X28XK71RuEXPapeyUOwNzg,5.0,10,4,7,"Wanting to help out the local economy, I thoug...",2020-04-23 18:26:29
...,...,...,...,...,...,...,...,...,...
967779,yhyJUBAJUG-9gEIQ7gXS_g,iZAAjkPZ0sopzfajcfdOUg,2PvPsZ3KRFCtHbQkNHvGpg,5.0,1,0,0,Ordered takeout and this place didn't disappoi...,2020-04-19 15:42:12
967780,5NxUmqweVTAZ_-FGJywTYw,AN8XscFSH1jctLSqAQ9-bA,-K0zTgGyxo-AeSkcV0IVaA,4.0,0,0,0,I took our annual managers meeting there for d...,2014-02-10 18:27:03
967781,-0rEInvO7q5cSI81NlH3-g,qtgODIPIsKsH2j3rItF9Tw,eZCt3doDaA-l4sg_OM67YQ,5.0,0,0,1,My favorite local coffee shop! Great drinks (i...,2019-07-24 14:19:52
967782,8icm1hiV87OZbDsyjQDLAQ,gu7nU1IM7U3lLwUbqLeiDQ,ww3YJXu5c18aGZXWmm00qg,5.0,0,0,0,We hit a quiet time here on a very busy weeken...,2011-11-08 06:38:05


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Cargar datos de entrenamiento
df_train = pd.read_csv("../data/train_reviews.csv")
df_train['text'] = df_train['text'].fillna("")
df_train['date'] = pd.to_datetime(df_train['date'])
df_train['year'] = df_train['date'].dt.year

# Seleccionar características adicionales de review, por ejemplo:
other_features = ['useful', 'funny', 'cool', 'year']
X_other_train = df_train[other_features].values

# Procesar el texto con TF-IDF en el training
tfidf = TfidfVectorizer(max_features=5000)
X_text_train_tfidf = tfidf.fit_transform(df_train['text'])

# Reducir la dimensionalidad del texto
svd = TruncatedSVD(n_components=50, random_state=42)
X_text_train_svd = svd.fit_transform(X_text_train_tfidf)

In [4]:
# Combinar las características numéricas con el embedding del texto
X_train = np.hstack([X_other_train, X_text_train_svd])
y_train = df_train['stars']

# Dividir el conjunto de entrenamiento en entrenamiento y validación
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# Entrenar el modelo RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42, verbose=1, n_jobs=-1)
model.fit(X_train_final, y_train_final)

# Predecir en el conjunto de validación para evaluar
y_pred_val = model.predict(X_val)
print("MAE en validación:", mean_absolute_error(y_val, y_pred_val))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  9.2min


In [ ]:
# Ahora, procesar el conjunto de test (sin ajustar de nuevo el vectorizador ni el SVD)
df_test = pd.read_csv('../data/test_reviews.csv')
df_test['text'] = df_test['text'].fillna("")
df_test['date'] = pd.to_datetime(df_test['date'])
df_test['year'] = df_test['date'].dt.year

X_other_test = df_test[other_features].values

# Transformar el texto de test con el vectorizador y el SVD ya ajustados
X_text_test_tfidf = tfidf.transform(df_test['text'])
X_text_test_svd = svd.transform(X_text_test_tfidf)

# Combinar las características
X_test = np.hstack([X_other_test, X_text_test_svd])

# Realizar las predicciones en el conjunto de test
predictions = model.predict(X_test)

# Crear el DataFrame con el formato requerido: review_id, stars
df_output = pd.DataFrame({
    'review_id': df_test['review_id'],
    'stars': predictions.round(1)  # Redondea a 1 decimal, como 4.0, 3.0, etc.
})

# Exportar el CSV con las predicciones
df_output.to_csv('test_review_predictions.csv', index=False)
print("Predicciones exportadas a 'test_review_predictions.csv'")